In [1]:
import sqlite3
import pandas as pd
import sys


sys.path.append('../')
from functions_env import DB_PATH


conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [2]:
df_works = pd.read_sql_query("SELECT * FROM individual_created_work", conn)
individuals = pd.read_sql_query("SELECT * FROM individuals_kept", conn)

df_works = df_works[df_works['individual_wikidata_id'].isin(list(set(individuals.individual_wikidata_id)))]

df_count_works = df_works.groupby('individual_wikidata_id')['work_wikidata_id'].count().reset_index()
df_count_works = df_count_works.sort_values('work_wikidata_id', ascending=False).reset_index(drop=True)

created_work = pd.read_sql_query("SELECT * FROM created_work", conn)
created_work = created_work[created_work['work_wikidata_id'].isin(list(set(df_works.work_wikidata_id)))]
created_work = created_work[['work_wikidata_id', 'instance_label']].drop_duplicates()

df_count_instance = created_work.instance_label.value_counts().rename('count_works').head(20).reset_index()
df_count_instance['percent'] = round(df_count_instance['count_works']/len(set(created_work.work_wikidata_id))*100, 1)
df_count_instance.columns = ['Type of work', 'N Works', '%']

df_count_instance

,Type of work,N Works,%
0,painting,239501,32.4
1,biographical article,95224,12.9
2,poem,58204,7.9
3,print,55973,7.6
4,literary work,42532,5.7
5,"version, edition, or translation",39430,5.3
6,encyclopedia article,28606,3.9
7,drawing,28424,3.8
8,article,19878,2.7
9,photograph,19779,2.7


In [3]:
len(set(df_works.work_wikidata_id))

740098

In [4]:
df_count_works.work_wikidata_id.median()

3.0